In [2]:
# !pip install crewai -r ./requirements.txt

In [3]:
!dir

abhimanyu_v1-Copy1.ipynb  README.md	    src        Untitled.ipynb
dev_llm			  requirements.txt  tutorials


In [3]:
import dotenv
assert dotenv.load_dotenv()

In [4]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
# MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
# MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.55})
# low_llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.1})
# high_llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.5})

In [3]:
questions = [
    "Visualize the number of students who participated in the PIRLS 2021 study per country"
]

In [5]:
response = llm.invoke(message)
print(response.content)

NameError: name 'message' is not defined

In [5]:
from src.static.submission import Submission
from src.static.util import PROJECT_ROOT
import src.submission.tools.database as db_tools
import src.submission.tools.chart as chart

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


2024-10-22 06:47:52,702 - 140217121097472 - font_manager.py-font_manager:1035 - WARNING: Matplotlib is building the font cache; this may take a moment.


In [6]:
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from langchain_core.tools import tool

In [7]:
@CrewBase
class AdvancedPIRLSCrew(Submission):
    """Data Analysis Crew for the GDSC project."""
    # Load the files from the config directory
    agents_config = PROJECT_ROOT / 'submission' / 'config' / 'agents.yaml'
    tasks_config = PROJECT_ROOT / 'submission' / 'config' / 'tasks.yaml'

    def __init__(self, llm):
        self.llm = llm
        # self.low_llm = low_llm
        # self.high_llm = high_llm

    def run(self, prompt: str) -> str:
        return self.crew().kickoff(inputs={'user_question': prompt}).raw

    @agent
    def lead_data_analyst(self) -> Agent:
        a = Agent(
            config=self.agents_config['lead_data_analyst'],
            llm=self.llm,
            allow_delegation=True,
            verbose=True
        )
        return a

    @agent
    def data_engineer(self) -> Agent:
        a = Agent(
            config=self.agents_config['data_engineer'],
            llm=self.llm,
            allow_delegation=False,
            verbose=True,
            tools=[
                db_tools.query_database,
                db_tools.get_possible_answers_to_question,
                db_tools.get_questions_of_given_type
            ]
        )
        return a

    @task
    def answer_question_task(self) -> Task:
        t = Task(
            config=self.tasks_config['answer_question_task'],
            agent=self.lead_data_analyst(),
            tools=[
                chart.generate_chart,
            ]
        )
        return t

    @crew
    def crew(self) -> Crew:
        """Creates the data analyst crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
            max_iter=5,
            cache=True
        )

In [3]:
crew = AdvancedPIRLSCrew(llm=llm)

print(crew.run("How do teacher's job satisfaction relate to student learning outcomes, as reflected by their level of reading proficiency?"))

NameError: name 'AdvancedPIRLSCrew' is not defined

In [46]:
# crew = AdvancedPIRLSCrew(llm=llm)

# print(crew.run("What was Turkey's average score for fourth graders in reading?"))

In [47]:
# crew = AdvancedPIRLSCrew(llm = llm)

# print(crew.run("How many students in Slovenia scored at the most advanced level in the PIRLS 2021 study?"))

In [50]:
# result_json = []
# for ques in questions[5:]:
#     print(crew.run(ques))
    

In [48]:
# crew = AdvancedPIRLSCrew(llm=llm)

# print(crew.run("How do the achievements of students from low socioeconomic status and more fluent families compare?"))

In [1]:
import requests

# Some wrapper code to send requests to our crew
def ask_question(question: str, url: str):
    data = {'prompt': question}
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, json=data, headers=headers)

    return response.json()

In [2]:
# note that the port needs to be set to 8000
REMOTE_HOST = 'http://52.200.159.60:8000//run'

In [9]:
questions = [
    "How do the achievements of students from low socioeconomic status and more fluent families compare?",
    "What is the global trend in fourth grade reading achievement as revealed by the PIRLS 2021 results?",
    "Which countries performed the best in the PIRLS 2021 study, and what common factors do they share in their educational systems?",
    "What insights can be drawn from student and teacher questionnaire responses regarding factors influencing reading achievement?",
    "Pick any country. How have the reading skills of students in this country evolved from previous PIRLS assessments to 2021?",
    "Visualize the number of students who participated in the PIRLS 2021 study per country.",
    "Show a plot of the correlation of a countries GDP and its readings skills according to the PIRLS 2021 study"
]

In [3]:
res = ask_question("What's the impact of early literacy to the future of the pupils?", REMOTE_HOST)
print(res['result'])

#GDSC7

# The Impact of Early Literacy on the Future of Pupils 📚🔍

Early literacy plays a crucial role in shaping the educational trajectory and future success of pupils. Based on the analysis of the PIRLS 2021 dataset, we can draw several important conclusions about the relationship between early literacy activities and later reading achievement. 

## Key Findings 🔑

1. **Frequency of Reading Books** 📖
   - Students whose parents read books with them "Often" before primary school consistently achieved higher average reading scores.
   - Example: In Albania, students read to "Often" scored 544.2, compared to 514.0 for "Sometimes."

2. **Early Writing Practice** ✍️
   - Frequent practice in writing letters or words before primary school generally correlated with better reading performance.
   - Example: In Abu Dhabi, UAE, students who "Often" wrote letters or words scored 514.4, compared to 465.4 for "Sometimes."

3. **Pre-primary Education** 🏫
   - Attendance in pre-primary education p

In [4]:
res = ask_question("How do instructions in a second language affect the reading performance of 4th graders in education systems participating in PIRLS 2021?", REMOTE_HOST)
res['result']

"#GDSC7\n\n# How do instructions in a second language affect the reading performance of 4th graders in education systems participating in PIRLS 2021? 📚🌍\n\nWhile the PIRLS 2021 dataset doesn't provide specific information on second language instruction, we can analyze the reading performance scores to draw some insights and consider the potential impact of language instruction policies. Here's a data-driven analysis based on the available information:\n\n## 📊 Key Findings\n\n1. **Performance Disparity**: There is a significant gap in reading performance between the top and bottom-performing education systems.\n\n   - Top performers: \n     1. Moscow City, Russian Federation (599.01)\n     2. Singapore (583.01)\n     3. Ireland (578.49)\n   \n   - Bottom performers:\n     1. South Africa (267.19)\n     2. Morocco (365.39)\n     3. Jordan (370.31)\n\n2. **Linguistic Diversity**: Many top-performing systems are known for their linguistic diversity or bilingual policies.\n\n   - Singapore:

In [69]:
res = ask_question(questions[2], REMOTE_HOST)
res['result']

"# Top Performing Countries in PIRLS 2021 and Their Educational System Factors 📚🌍\n\nThe Progress in International Reading Literacy Study (PIRLS) 2021 results reveal interesting insights about the countries that excel in fourth-grade reading skills. Let's explore the top performers and the common factors in their educational systems.\n\n## Top Performers 🏆\n\nHere are the top 8 countries/regions by average reading achievement score:\n\n| Rank | Country/Region | Average Reading Score |\n|------|----------------|------------------------|\n| 1 | Moscow City, Russian Federation | 599.01 |\n| 2 | Singapore | 583.01 |\n| 3 | Hong Kong SAR | 574.54 |\n| 4 | Russian Federation | 571.96 |\n| 5 | Northern Ireland | 567.03 |\n| 6 | Croatia | 559.34 |\n| 7 | United Kingdom | 556.96 |\n| 8 | Quebec, Canada | 553.74 |\n\n## Common Factors in Educational Systems 🔍\n\nAnalyzing the data from these top-performing countries/regions, we can identify several common factors and interesting trends:\n\n1. **

In [59]:
res = ask_question(questions[3], REMOTE_HOST)
res['result']

'# Insights from PIRLS 2021: Factors Influencing Reading Achievement 📚🔍\n\nBased on the analysis of student and teacher questionnaire responses from the PIRLS 2021 dataset, several key insights can be drawn regarding factors influencing reading achievement among fourth-grade students. Let\'s explore these findings in detail:\n\n## 1. Gender Differences in Reading Achievement 👧👦\n\nOur analysis reveals a notable gender gap in reading performance:\n\n| Gender | Average Reading Score |\n|--------|------------------------|\n| Female | 504.62                 |\n| Male   | 485.40                 |\n| Other  | 525.09                 |\n\n🔑 Key Insight: Girls outperform boys in reading by an average of 19.22 points. This gender gap is consistent with previous PIRLS cycles and other international assessments, suggesting that targeted interventions to support boys\' reading development may be necessary.\n\n## 2. Impact of Home Library Size 📚🏠\n\nThe number of books at home shows a strong positiv

In [68]:
res = ask_question(questions[4], REMOTE_HOST)
res['result']

"# Singapore's Reading Skills in PIRLS 2021: A Comprehensive Analysis 📚🇸🇬\n\nWhile we don't have direct access to historical PIRLS data for Singapore, we can provide a detailed analysis of the country's performance in the 2021 assessment. This analysis will give us valuable insights into the current state of reading skills among Singaporean fourth-grade students.\n\n## Overall Performance 🏆\n\nSingapore demonstrated exceptional reading skills in PIRLS 2021, with an impressive average score of 583.02. This score is significantly above the PIRLS scale centerpoint of 500, indicating that Singaporean students are performing at a high level in reading literacy.\n\n## Reading Purposes and Processes 📊\n\n| Reading Aspect | Average Score |\n|----------------|---------------|\n| Literary Experience | 587.39 |\n| Acquiring and Using Information | 582.33 |\n| Interpreting and Evaluating | 586.57 |\n| Retrieving and Straightforward Inferencing | 579.90 |\n\nSingaporean students showed strength acr

In [ ]:
res = ask_question(questions[5], REMOTE_HOST)
res['result']

In [67]:
res = ask_question("Visualise the student to teacher ratio country wise and limit it to top 8", REMOTE_HOST)
res['result']

"# Correlation between Countries' GDP and Reading Skills in PIRLS 2021 📚💰\n\nBased on the analysis of the PIRLS 2021 study data, we can provide insights into the relationship between countries' GDP and their fourth-grade students' reading skills. While we don't have direct access to GDP data, we can make some general observations and discuss the implications of the results.\n\n## Key Findings 🔍\n\n1. **Top Performers**: The countries with the highest reading achievement scores in PIRLS 2021 include:\n\n   1. Singapore (583.0061)\n   2. Ireland (578.4871)\n   3. Russian Federation (571.9641)\n   4. Croatia (559.3354)\n   5. United Kingdom (556.9566)\n\n   These countries are generally known to have relatively high GDP per capita, suggesting a possible positive correlation between economic strength and reading performance.\n\n2. **Lower Performers**: Countries with lower scores include:\n\n   49. Egypt (377.0759)\n   50. Jordan (370.3101)\n   51. Morocco (365.3946)\n   52. South Africa (